### **Caso Técnico: Data Engineer**  - Miguel Ángel Flores Álvarez

### **Instalación de paquetes**

In [1]:
! pip install pandas requests 

### **Librerías**

In [2]:
import pandas as pd
import requests
from typing import List, Dict, Optional

### **Solución**

In [3]:
class Indicadores:
    '''
    Clase para obtener la información de países, incluyendo la población, moneda y región, usando APIs de World Bank y BDC
    '''
    llave = 'bdc_eca00bfa2d0b4c7e8f3f5ff81b63e0b2'
    url_api_poblacion = 'https://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL'
    url_api_moneda = 'http://api-bdc.net/data/country-info'
    url_api_paises = 'https://api.worldbank.org/v2/country?format=json&per_page=400'

    # Obtencion de paises 'validos' y registro de regiones
    def __init__(self):
        '''
        --------------------------------------------------------------------------------------------------------------------
        Constructor __init__
        --------------------------------------------------------------------------------------------------------------------
        Descripción: 
            Obtiene la lista de países válidos y crea un diccionario con ISO3 como clave, que contiene nombre y región del 
            país.

        Inputs: 
            Ninguno

        Outputs: 
            None
        --------------------------------------------------------------------------------------------------------------------
        '''

        # Generar peticion a API de información de paises
        registro_pais = requests.get(self.url_api_paises)
        # En caso de error 
        registro_pais.raise_for_status()
        # Conversion de json a objeto de python
        resultado = registro_pais.json()
        # Donde se genera una lista de dos elementos, el 
        ## data[0] → metadatos (ej. número de páginas, total de países, etc.).
        ## data[1] → lista de países (cada uno como un diccionario).
        paises = resultado[1]

        # Construcción de diccionario con base al ISO3, donde establece el Nombre y la Region
        self.informacion_paises = {
            pais['id']:{
                'Nombre': pais['name'],
                'Region': pais['region']['value']
            }
            # Si es una region no aplica
            for pais in paises if pais['region']['id'] != 'NA'
        } 

    def moneda(self, iso3: str) -> str:
        '''
        --------------------------------------------------------------------------------------------------------------------
        Función moneda
        --------------------------------------------------------------------------------------------------------------------
        Descripción:
            Obtiene el código de moneda de un país según su ISO3 mediante API de BDC.

        Inputs:
            iso3 (str): Código ISO3 del país.

        Outputs:
            str: Código de moneda (ej. 'USD') o 'Desconocida' si falla la petición o no existe.
        --------------------------------------------------------------------------------------------------------------------
        '''
        # Validación del tipo de entrada
        if not isinstance(iso3, str):
            raise TypeError('ISO3 debe ser un string')
        
        # URL Dinámico
        url_actualizado_moneda = f'{self.url_api_moneda}?code={iso3.lower()}&key={self.llave}'
        
        try:
            # Llama al API
            registro_moneda = requests.get(url_actualizado_moneda)
            # Valida el HTTP
            registro_moneda.raise_for_status()
            # Parsea el JSON
            resultado = registro_moneda.json()
            # Obtención del tipo de moneda
            iso_moneda = resultado.get('currency', {})
            return iso_moneda.get('code', 'Desconocida')
        
        except:
            # En caso de que falle
            return 'Desconocida'

    def poblacion(self, top: int, fecha: Optional[int] = None) -> pd.DataFrame:
        '''
        --------------------------------------------------------------------------------------------------------------------
        Función poblacion
        --------------------------------------------------------------------------------------------------------------------
        Descripción:
            Obtiene la población de los países válidos y su información asociada.
            Puede solicitar un año específico o el valor más reciente.

        Inputs:
            top (int): Número de países a retornar según población descendente.
            fecha (Optional[int]): Año específico; si None, obtiene el valor más reciente.

        Outputs:
            pd.DataFrame: DataFrame con columnas ['Pais', 'ISO3', 'Region', 'Poblacion', 'Moneda', 'Año'].
        --------------------------------------------------------------------------------------------------------------------
        '''
        # Validación del tipo de entrada
        if not isinstance(top, int):
            raise TypeError('La cantidad de países a determinar debe ser entero')

        if fecha and not isinstance(fecha, int):
            raise TypeError('La fecha debe ser entero o None')

        # Si establecen el valor de la fecha
        if fecha:
            url = f'{self.url_api_poblacion}?format=json&per_page=300&date={fecha}'
        # Caso contrario, valor más reciente
        else:
            url = f'{self.url_api_poblacion}?format=json&per_page=300&most_recent_value=true'
        
        # Llamada de API
        registro_poblacion = requests.get(url)
        # Validación HTTP
        registro_poblacion.raise_for_status()
        # Parsea JSON
        resultado = registro_poblacion.json()

        datos = []

        if resultado and len(resultado) > 1:
            # Extraccion de lista de observaciones
            paises = resultado[1] 
            for pais in paises:
                iso3 = pais['countryiso3code']
                # Corrobora la existencia de la poblacion 'value' y que el ISO3 este dentro de paises validos
                if pais.get('value') is not None and iso3 in self.informacion_paises:
                    # Llama a la función moneda para la obtención de moneda por iso3
                    moneda = self.moneda(iso3)
                    datos.append({
                        'Pais': self.informacion_paises[iso3]['Nombre'],
                        'ISO3': iso3,
                        'Region': self.informacion_paises[iso3]['Region'],
                        'Poblacion': int(pais['value']),
                        'Moneda': moneda,
                        'Año': pais['date']
                    })
        df = pd.DataFrame(datos)
        if not df.empty:
            df = df.sort_values('Poblacion', ascending=False).head(top)
        return df.reset_index(drop = True)

In [4]:
class CambioMXN:
    '''
    Clase para convertir la población de países a pesos mexicanos (MXN), usando tasas de cambio desde Fixer.io,
    con respaldo de RestCountries y un diccionario manual para monedas raras.
    '''

    url = 'http://data.fixer.io/api/latest'
    llave = '1c3233f57ed4f5ad1a2580c3145c2b5b'
    url_respaldo = 'https://restcountries.com/v3.1/alpha/'

    respaldo_monedas = {
        'TRY': 18.52,
        'TZS': 2702.70,
    }

    def __init__(self):
        '''
        --------------------------------------------------------------------------------------------------------------------
        Constructor __init__
        --------------------------------------------------------------------------------------------------------------------
        Descripción:
            Inicializa la clase y establece el formato de visualización de pandas para redacción numérica.

        Inputs:
            Ninguno
        
        Outputs:
            None
        --------------------------------------------------------------------------------------------------------------------
        '''
        pd.options.display.float_format = '{:,.0f}'.format
    
    def _tasa_moneda(self, symbols: Optional[List[str]] = None) -> Dict[str, float]:
        '''
        --------------------------------------------------------------------------------------------------------------------
        Función _tasa_moneda
        --------------------------------------------------------------------------------------------------------------------
        Descripción:    
            Obtiene los tipos de cambio de monedas respecto a EUR desde la API de Fixer.io
            Limita la consulta a monedas específicas
        
        Inputs:
            symbols(Optional[Lits[str]]): Lista de códigos de moneda a consultar. Si es None, obtiene las disponibles.

        Outputs:
            dict: Diccionario con monedas como llaves y tasas respecto a EUR como valor
                Ejemplo: {'USD': 1.12, 'MXN': 23.45}
        --------------------------------------------------------------------------------------------------------------------
        '''
        # Validación del tipo de entrada
        if symbols is not None and not isinstance(symbols, list):
            raise TypeError('Symbols debe contener una lista de strings o None')
        
        # Limitación de monedas, al establecer un listado de estas y no todas
        parametros = {'access_key': self.llave}
        if symbols:
            parametros['symbols'] = ','.join(symbols)
        
        # Llamada API
        registro_moneda = requests.get(self.url, params=parametros)
        # Validación HTTP
        registro_moneda.raise_for_status()
        # Parsea JSON
        resultado = registro_moneda.json()

        if not resultado.get('success', False):
            raise ValueError(f'Error Fixer: {resultado}')
        
        # Devielve el diccionario de tipos de cambio, REFERIDOS POR EUR
        return resultado['rates']
    
    def _moneda_respaldo(self, iso3: str) -> str:
        '''
        --------------------------------------------------------------------------------------------------------------------
        Función _moneda_respaldo
        --------------------------------------------------------------------------------------------------------------------
        Descripción:
            Obtiene el código ISO de moneda de un país mediante la API RestCountries.
            * Respaldo cuando la moneda no está disponible en Fixer o es desconocida.
        
        Inputs:
            iso3 (str): Código ISO3 del país.
        
        Outputs: 
            str: Código ISO de la moneda (ej. 'TRY'), o 'Desconocida' si no se encuentra.
        --------------------------------------------------------------------------------------------------------------------
        '''
        # Validación del tipo de entrada
        if not isinstance(iso3, str):
            raise TypeError('ISO3 debe ser un string')
         
        try:
            # Llama al API
            resp = requests.get(f'{self.url_respaldo}{iso3}')
            # Valida el HTTP
            resp.raise_for_status()
            # Parsea el JSON
            datos = resp.json()
            # Obtención del tipo de moneda
            monedas = list(datos[0].get('currencies', {}).keys())
            return monedas[0] if monedas else 'Desconocida'
        except Exception as e:
            # Mensaje de error indicando fallo en iso3 y descripción
            print(f'Error RestCountries para {iso3}: {e}')
            # EN caso de que falle
            return 'Desconocida'
    
    def conversion(self, dataset: pd.DataFrame) -> pd.DataFrame:
        '''
        --------------------------------------------------------------------------------------------------------------------
        Función conversion
        --------------------------------------------------------------------------------------------------------------------
        Descripción:
            Convierte la población de cada país a pesos mexicanos (MXN), considerando 1 centavo por persona.
            Usa primero tasas de Fixer.io, luego RestCountries y finalmente un diccionario de respaldo para monedas extraordinarias.

        Inputs:
            dataset (pd.DataFrame): DataFrame con columnas mínimas ['Pais', 'ISO3', 'Poblacion', 'Moneda'].
                                     - 'Pais': Nombre del país
                                     - 'ISO3': Código ISO3 del país
                                     - 'Poblacion': Número de habitantes
                                     - 'Moneda': Código ISO de la moneda (puede estar vacío)
        Outputs:
            pd.DataFrame: DataFrame con columnas ['Pais', 'ISO3', 'Poblacion', 'Moneda', 'MXN_total']:
                          - 'Poblacion': formateada con separador de miles
                          - 'MXN_total': valor en pesos mexicanos (formateado), o 'No disponible' si no hay tasa
        --------------------------------------------------------------------------------------------------------------------
        '''
        # Validación del tipo de entrada
        if not isinstance(dataset, pd.DataFrame):
            raise TypeError('El conjunto debe ser DataFrame')
        if 'Moneda' not in dataset.columns:
            dataset['Moneda'] = 'Desconocida'

        # Listado de monedas únicas en el df
        monedas = dataset['Moneda'].unique().tolist()
        # Asgurar que MXN este presente para calculvar conversión
        if 'MXN' not in monedas:
            monedas.append('MXN')

        # Tasas de cambio
        tasas = self._tasa_moneda(symbols=monedas)

        resultados = []
        for _, fila in dataset.iterrows():
            iso3 = fila['ISO3']
            moneda = fila['Moneda']
            poblacion = fila['Poblacion']

            # Primer respaldo
            if moneda == 'Desconocida' or moneda not in tasas:
                moneda = self._moneda_respaldo(iso3)

            if moneda not in tasas:
                # Segundo respaldo
                if moneda in self.respaldo_monedas:
                    tasa_moneda = self.respaldo_monedas[moneda]  # 1 EUR = X MONEDA
                    cambio_mxn = tasas['MXN'] / tasa_moneda
                    total_mxn = poblacion * 0.01 * cambio_mxn
                else:
                    cambio_mxn = None
                    total_mxn = None
            # Conversión
            else:
                cambio_mxn = tasas['MXN'] / tasas[moneda]
                total_mxn = poblacion * 0.01 * cambio_mxn

            resultados.append({
                'Pais': fila['Pais'],
                'ISO3': iso3,
                'Poblacion': poblacion,
                'Moneda': moneda,
                'Tipo_cambio': cambio_mxn,
                'MXN_total': total_mxn
            })

        # Construcción de df
        df_resultados = pd.DataFrame(resultados)
        # Presentación columnas
        df_resultados['Poblacion'] = df_resultados['Poblacion'].apply(lambda x: f'{int(x):,}')
        df_resultados['MXN_total'] = df_resultados['MXN_total'].apply(
            lambda x: f'{x:,.2f}' if x is not None else 'No disponible'
        )
        df_resultados['Tipo_cambio'] = df_resultados['Tipo_cambio'].apply(
            lambda x: f'{x:,.4f}' if x is not None else 'No disponible'
        )
        return df_resultados

#### **Proceso de aplicación**

In [5]:
if __name__ == '__main__':
    # Inicialización de lista de países válidos y sus regiones mediante API World Bank
    ind = Indicadores()
    # Llamada del método de poblacion de Indicadores, con el cual obtienes n paises con mayor población en x fecha
    df_top = ind.poblacion(top=30, fecha=2021)
    # Creación de la instancia que habilita el cambio de moneda
    conv = CambioMXN()
    # Llamada del método de conversion de CambiomXN, con el cual convierte la moneda a MXN, generando un nuevo DataFrame
    df_mxn = conv.conversion(df_top)
    print('==== Conversión a MXN ====')
    print(df_mxn)

==== Conversión a MXN ====
                  Pais ISO3      Poblacion Moneda Tipo_cambio      MXN_total
0                India  IND  1,414,203,896    INR      0.2131   3,013,478.84
1                China  CHN  1,412,360,000    CNY      2.6116  36,884,955.97
2        United States  USA    332,099,760    USD     18.6804  62,037,498.38
3            Indonesia  IDN    276,758,053    IDR      0.0011       3,158.14
4             Pakistan  PAK    239,477,801    PKR      0.0663     158,706.44
5              Nigeria  NGA    218,529,286    NGN      0.0122      26,568.42
6               Brazil  BRA    209,550,294    BRL      3.4473   7,223,754.02
7           Bangladesh  BGD    167,658,854    BDT      0.1535     257,389.47
8   Russian Federation  RUS    144,746,762    RUB      0.2325     336,516.90
9               Mexico  MEX    127,648,148    MXN      1.0000   1,276,481.48
10               Japan  JPN    125,681,593    JPY      0.1267     159,266.58
11            Ethiopia  ETH    122,138,588    ETB